In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline

In [ ]:
gdata = pd.read_csv('../results/goodness.csv')
print(gdata.shape)
gdata.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.array(gdata['ai']),
                np.array(gdata['aj']),
                np.array(gdata['goodness']))
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.array(gdata['ai']),
                np.array(gdata['aj']),
                np.array(gdata['goodness']))
ax.view_init(45, 45)
plt.show()

In [ ]:
np.array(gdata['ai']).reshape(21, 21)

In [ ]:
np.array(gdata['ai']).shape

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(np.array(gdata['ai']),
                np.array(gdata['aj']),
                np.array(gdata['goodness']), rstride=8, cstride=8, alpha=0.3)
plt.show()

In [ ]:
np.array(gdata['goodness']).reshape(21, 21)

In [ ]:
a = np.array(gdata['ai']).reshape(21, 21)
a

In [ ]:
b = np.array(gdata['aj']).reshape(21, 21)
b

In [ ]:
c = np.array(gdata['goodness']).reshape(21, 21)
c

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(a, b, c, rstride=1, cstride=1, alpha=0.3)
ax.view_init(0, 90)
plt.show()

In [ ]:
# %load http://matplotlib.org/mpl_examples/mplot3d/wire3d_demo.py
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a, b, c, rstride=1, cstride=1)
ax.view_init(20, 140)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(a, b, c, rstride=1, cstride=1, alpha=0.3)
ax.view_init(45, 45)
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm

fig = plt.figure(figsize=(25,25))
ax = fig.gca(projection='3d')

ax.plot_surface(a, b, c, rstride=1, cstride=1, alpha=0.3)
cset = ax.contourf(a, b, c, zdir='z', offset=-100, cmap=cm.coolwarm)
cset = ax.contourf(a, b, c, zdir='x', offset=-40, cmap=cm.coolwarm)
cset = ax.contourf(a, b, c, zdir='y', offset=40, cmap=cm.coolwarm)

ax.view_init(20, 120)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import os, sys
import numpy as np


##### TO CREATE A SERIES OF PICTURES

def make_views(ax,angles,elevation=None, width=4, height = 3,
               prefix='tmprot_',**kwargs):
    """
    Makes jpeg pictures of the given 3d ax, with different angles.
    Args:
        ax (3D axis): te ax
        angles (list): the list of angles (in degree) under which to
                       take the picture.
        width,height (float): size, in inches, of the output images.
        prefix (str): prefix for the files created.

    Returns: the list of files created (for later removal)
    """

    files = []
    ax.figure.set_size_inches(width,height)

    for i,angle in enumerate(angles):

        ax.view_init(elev = elevation, azim=angle)
        fname = '%s%03d.jpeg'%(prefix,i)
        ax.figure.savefig(fname)
        files.append(fname)

    return files



##### TO TRANSFORM THE SERIES OF PICTURE INTO AN ANIMATION

def make_movie(files,output, fps=10,bitrate=1800,**kwargs):
    """
    Uses mencoder, produces a .mp4/.ogv/... movie from a list of
    picture files.
    """

    output_name, output_ext = os.path.splitext(output)
    command = { '.mp4' : 'mencoder "mf://%s" -mf fps=%d -o %s.mp4 -ovc lavc\
                         -lavcopts vcodec=msmpeg4v2:vbitrate=%d'
                %(",".join(files),fps,output_name,bitrate)}

    command['.ogv'] = command['.mp4'] + '; ffmpeg -i %s.mp4 -r %d %s'%(output_name,fps,output)

    print(command[output_ext])
    output_ext = os.path.splitext(output)[1]
    os.system(command[output_ext])


def make_gif(files,output,delay=100, repeat=True,**kwargs):
    """
    Uses imageMagick to produce an animated .gif from a list of
    picture files.
    """

    loop = -1 if repeat else 0
    os.system('convert -delay %d -loop %d %s %s'
              %(delay,loop," ".join(files),output))


def make_strip(files,output,**kwargs):
    """
    Uses imageMagick to produce a .jpeg strip from a list of
    picture files.
    """

    os.system('montage -tile 1x -geometry +0+0 %s %s'%(" ".join(files),output))


##### MAIN FUNCTION

def rotanimate(ax, angles, output, **kwargs):
    """
    Produces an animation (.mp4,.ogv,.gif,.jpeg,.png) from a 3D plot on
    a 3D ax

    Args:
        ax (3D axis): the ax containing the plot of interest
        angles (list): the list of angles (in degree) under which to
                       show the plot.
        output : name of the output file. The extension determines the
                 kind of animation used.
        **kwargs:
            - width : in inches
            - heigth: in inches
            - framerate : frames per second
            - delay : delay between frames in milliseconds
            - repeat : True or False (.gif only)
    """

    output_ext = os.path.splitext(output)[1]

    files = make_views(ax,angles, **kwargs)

    D = { '.mp4' : make_movie,
          '.ogv' : make_movie,
          '.gif': make_gif ,
          '.jpeg': make_strip,
          '.png':make_strip}

    D[output_ext](files,output,**kwargs)

    #for f in files:
    #    os.remove(f)

##### EXAMPLE

if __name__ == '__main__':

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    X, Y, Z = axes3d.get_test_data(0.05)
    s = ax.plot_surface(X, Y, Z, cmap=cm.jet)
    plt.axis('off') # remove axes for visual appeal

    angles = np.linspace(0,360,21)[:-1] # Take 20 angles between 0 and 360

    # create an animated gif (20ms between frames)
    # rotanimate(ax, angles,'movie_sample.gif',delay=20)

    # create a movie with 10 frames per seconds and 'quality' 2000
    # rotanimate(ax, angles,'movie_sample.mp4',fps=10,bitrate=2000)

    # create an ogv movie
    # rotanimate(ax, angles, 'movie_sample.ogv',fps=10)

In [ ]:
#import numpy as np
#angles = np.linspace(0,360,21)[:-1] # A list of 20 angles between 0 and 360
 
# create an animated gif (20ms between frames)
#rotanimate(ax, angles,'movie.gif',delay=20) 
 
# create a movie with 10 frames per seconds and 'quality' 2000
#rotanimate(ax, angles,'movie.mp4',fps=10,bitrate=2000)
 
# create an ogv movie
#rotanimate(ax, angles, 'movie.ogv',fps=10) 

In [ ]:
# %load http://matplotlib.org/mpl_examples/mplot3d/wire3d_demo.py
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(a, b, c, rstride=1, cstride=1)

plt.show()

rotanimate(ax, angles,'rotate1.gif', delay=40, width=11, height = 8.5)

In [ ]:
%%bash
rm out.mp4
ffmpeg -framerate 1/0.5 -i tmprot_%03d.jpeg -c:v libx264 -vf fps=25 -pix_fmt yuv420p out.mp4

In [ ]:
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
# from matplotlib import cm

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.plot_surface(a, b, c, alpha=0.3)
# cset = ax.contourf(a, b, c, zdir='c', offset=0, cmap=cm.coolwarm)
# cset = ax.contourf(a, b, c, zdir='b', offset=0, cmap=cm.coolwarm)
# cset = ax.contourf(a, b, c, zdir='a', offset=0, cmap=cm.coolwarm)

# ax.set_xlabel('a')
# ax.set_xlim(0, 1)
# ax.set_ylabel('b')
# ax.set_ylim(0, 1)
# ax.set_zlabel('c')
# ax.set_zlim(0, 2)

# plt.show()

In [ ]:
# # %load http://matplotlib.org/mpl_examples/mplot3d/contourf3d_demo2.py
# """
# .. versionadded:: 1.1.0
#    This demo depends on new features added to contourf3d.
# """

# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
# from matplotlib import cm

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# X, Y, Z = axes3d.get_test_data(0.05)
# ax.plot_surface(X, Y, Z, rstride=8, cstride=8, alpha=0.3)
# cset = ax.contourf(X, Y, Z, zdir='z', offset=-100, cmap=cm.coolwarm)
# cset = ax.contourf(X, Y, Z, zdir='x', offset=-40, cmap=cm.coolwarm)
# cset = ax.contourf(X, Y, Z, zdir='y', offset=40, cmap=cm.coolwarm)

# ax.set_xlabel('X')
# ax.set_xlim(-40, 40)
# ax.set_ylabel('Y')
# ax.set_ylim(-40, 40)
# ax.set_zlabel('Z')
# ax.set_zlim(-100, 100)

# plt.show()